In [1]:
!pip  install pm4py
!pip install python-Levenshtein

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.3 MB/s eta 0:00:0000:0100:01
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=d82a84e873a8ba5baf9e801ed04f17153288794438646fc9fe1d5253525668f8
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00a 0:00:01


In [6]:
import pm4py
import numpy as np
import cupy as cp
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from sklearn.manifold import MDS
import json
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from collections import Counter
import time
import math
from numba import cuda

# Multi-GPU setup
n_gpus = cp.cuda.runtime.getDeviceCount()
print(f"Running: Using {n_gpus} GPU(s)...")

# Load .xes file
log_path = "/kaggle/input/event-log/RequestForPayment.xes"
print("Running: Loading .xes file...")
start_time = time.time()
log = xes_importer.apply(log_path)
traces = [tuple(event["concept:name"] for event in trace) for trace in log]
unique_activities = sorted(set().union(*[set(trace) for trace in traces]))
n_traces = len(traces)
print(f"Completed loading: {time.time() - start_time:.2f}s (Traces: {n_traces})")

# A4: Generic Edit Distance
def get_3grams(traces, activities):
    activity_to_idx = {a: i for i, a in enumerate(activities)}
    g3_counts = cp.zeros((len(activities), len(activities), len(activities)), dtype=cp.int32)
    for trace in traces:
        for i in range(len(trace) - 2):
            x, a, y = trace[i:i+3]
            g3_counts[activity_to_idx[x], activity_to_idx[a], activity_to_idx[y]] += 1
    return g3_counts

def compute_substitution_scores_multi_gpu(traces, activities):
    print("Running: Computing Substitution Scores (Algorithm 1)...")
    start_time = time.time()
    n_activities = len(activities)
    g3_counts = get_3grams(traces, activities)
    Xa = [set() for _ in range(n_activities)]
    for x_idx in range(n_activities):
        for a_idx in range(n_activities):
            for y_idx in range(n_activities):
                if g3_counts[x_idx, a_idx, y_idx] > 0:
                    Xa[a_idx].add((x_idx, y_idx))
    C = cp.zeros((n_activities, n_activities), dtype=cp.float32)
    for a_idx in range(n_activities):
        for b_idx in range(n_activities):
            if a_idx == b_idx:
                counts = g3_counts[:, a_idx, :]
                C[a_idx, a_idx] = cp.sum(counts * (counts - 1) / 2)
            else:
                Xab = Xa[a_idx] & Xa[b_idx]
                for x_idx, y_idx in Xab:
                    C[a_idx, b_idx] += g3_counts[x_idx, a_idx, y_idx] * g3_counts[x_idx, b_idx, y_idx]
    NC = cp.sum(C)
    M = C / NC
    pa = cp.zeros(n_activities)
    for a_idx in range(n_activities):
        pa[a_idx] = M[a_idx, a_idx] + cp.sum(M[a_idx, :]) - M[a_idx, a_idx]
    E = cp.zeros((n_activities, n_activities))
    for a_idx in range(n_activities):
        for b_idx in range(n_activities):
            if a_idx == b_idx:
                E[a_idx, b_idx] = pa[a_idx]**2
            else:
                E[a_idx, b_idx] = 2 * pa[a_idx] * pa[b_idx]
    S = cp.log2(M / E)
    S = cp.where(cp.isinf(S) | cp.isnan(S), 0, S)
    print(f"Completed Substitution Scores: {time.time() - start_time:.2f}s")
    print(f"sub_scores: min={cp.min(S):.4f}, max={cp.max(S):.4f}, mean={cp.mean(S):.4f}")
    return S

def compute_insertion_scores_multi_gpu(traces, activities):
    print("Running: Computing Insertion Scores (Algorithm 2)...")
    start_time = time.time()
    n_activities = len(activities)
    g3_counts = get_3grams(traces, activities)
    Xa = [set() for _ in range(n_activities)]
    for x_idx in range(n_activities):
        for a_idx in range(n_activities):
            for y_idx in range(n_activities):
                if g3_counts[x_idx, a_idx, y_idx] > 0:
                    Xa[a_idx].add((x_idx, y_idx))
    count_right = cp.zeros((n_activities, n_activities), dtype=cp.float32)
    for a_idx in range(n_activities):
        for x_idx in range(n_activities):
            count_right[a_idx, x_idx] = cp.sum(g3_counts[x_idx, a_idx, :])
    norm = cp.sum(count_right, axis=1)
    pa = norm / cp.sum(norm)
    norm_count_right = count_right / norm[:, cp.newaxis]
    norm_count_right = cp.where(cp.isnan(norm_count_right), 0, norm_count_right)
    ins_scores = cp.log2(norm_count_right / (pa[:, cp.newaxis] * pa[cp.newaxis, :]))
    ins_scores = cp.where(cp.isinf(ins_scores) | cp.isnan(ins_scores), 0, ins_scores)
    print(f"Completed Insertion Scores: {time.time() - start_time:.2f}s")
    print(f"ins_scores: min={cp.min(ins_scores):.4f}, max={cp.max(ins_scores):.4f}, mean={cp.mean(ins_scores):.4f}")
    return ins_scores

@cuda.jit
def edit_distance_kernel(s_batch, t_batch, sub_scores, ins_scores, act_to_idx, distances):
    i = cuda.grid(1)
    if i < s_batch.shape[0] * t_batch.shape[0]:
        s_idx = i // t_batch.shape[0]
        t_idx = i % t_batch.shape[0]
        s_len = s_batch[s_idx, 0]
        t_len = t_batch[t_idx, 0]
        s = s_batch[s_idx, 1:int(s_len+1)]
        t = t_batch[t_idx, 1:int(t_len+1)]
        dp = cuda.local.array((100, 100), dtype=cp.float32)
        for m in range(int(s_len) + 1):
            for n in range(int(t_len) + 1):
                if m == 0 and n == 0:
                    dp[m, n] = 0
                elif m == 0:
                    dp[m, n] = dp[m, n-1] + max(1.0, abs(ins_scores[int(t[n-1]), int(t[n-1])]))
                elif n == 0:
                    dp[m, n] = dp[m-1, n] + max(1.0, abs(ins_scores[int(s[m-1]), int(s[m-1])]))
                else:
                    sub_cost = 0 if s[m-1] == t[n-1] else max(1.0, abs(sub_scores[int(s[m-1]), int(t[n-1])]))
                    del_cost = max(1.0, abs(ins_scores[int(s[m-1]), int(s[m-1])]))
                    ins_cost = max(1.0, abs(ins_scores[int(t[n-1]), int(t[n-1])]))
                    dp[m, n] = min(dp[m-1, n-1] + sub_cost, dp[m-1, n] + del_cost, dp[m, n-1] + ins_cost)
        distances[s_idx, t_idx] = dp[int(s_len), int(t_len)]

def generic_edit_distance_matrix(traces, sub_scores, ins_scores, activities, chunk_size=10000):
    print("Running: Computing Generic Edit Distance matrix on multi-GPU (A4)...")
    start_time = time.time()
    n_chunks = math.ceil(n_traces / chunk_size)
    max_len = max(max(len(t) for t in traces), 100)
    act_to_idx = {act: idx for idx, act in enumerate(activities)}
    traces_padded = cp.zeros((n_traces, max_len + 1), dtype=cp.float32)
    for i, trace in enumerate(traces):
        traces_padded[i, 0] = len(trace)
        for j, act in enumerate(trace):
            traces_padded[i, j + 1] = act_to_idx[act]
    sub_scores_gpu = cp.asarray(sub_scores)
    ins_scores_gpu = cp.asarray(ins_scores)

    def process_chunk(i, gpu_id):
        with cp.cuda.Device(gpu_id):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, n_traces)
            s_batch = traces_padded[start_idx:end_idx]
            t_batch = traces_padded
            distances = cp.zeros((s_batch.shape[0], t_batch.shape[0]), dtype=cp.float32)
            threads_per_block = 256
            blocks_per_grid = math.ceil((s_batch.shape[0] * t_batch.shape[0]) / threads_per_block)
            edit_distance_kernel[blocks_per_grid, threads_per_block](
                s_batch, t_batch, sub_scores_gpu, ins_scores_gpu, cp.array(list(act_to_idx.values())), distances
            )
            return distances

    dist_matrix_chunks = []
    for i in range(n_chunks):
        dist_matrix_chunks.append(process_chunk(i, i % n_gpus))
        elapsed = time.time() - start_time
        progress = (i + 1) / n_chunks
        eta = elapsed / progress * (1 - progress)
        print(f"A4 Distance Matrix: Processed {i+1}/{n_chunks} chunks ({progress*100:.1f}%), ETA: {eta:.2f}s")
    
    dist_matrix = cp.vstack(dist_matrix_chunks)
    dist_matrix_full = cp.zeros((n_traces, n_traces), dtype=cp.float32)
    dist_matrix_full[:dist_matrix.shape[0], :] = dist_matrix
    dist_matrix_full.T[:dist_matrix.shape[0], :] = dist_matrix
    dist_matrix_full = cp.maximum(dist_matrix_full, 0)
    print(f"Completed Distance Matrix: {time.time() - start_time:.2f}s")
    return dist_matrix_full.get()

# Phân cụm bằng DBSCAN
def cluster_traces(dist_matrix):
    print("Running: DBSCAN clustering...")
    start_time = time.time()
    eps = np.mean(dist_matrix[dist_matrix > 0]) * 0.05
    clustering = SklearnDBSCAN(eps=eps, min_samples=5, metric="precomputed")
    labels = clustering.fit_predict(dist_matrix)
    n_clusters = len(np.unique(labels)) - (1 if -1 in labels else 0)
    print(f"Completed DBSCAN: {time.time() - start_time:.2f}s (Clusters: {n_clusters})")
    return labels, n_clusters

# Tính Silhouette Index
def compute_silhouette_index(dist_matrix, labels):
    print("Running: Computing Silhouette Index...")
    start_time = time.time()
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
    if n_clusters < 2:
        print("Warning: Need at least 2 clusters for meaningful Silhouette Index.")
        return -1
    silhouette_scores = []
    for i in range(len(labels)):
        if labels[i] == -1: continue
        same_cluster = (labels == labels[i])
        a_x = np.mean(dist_matrix[i, same_cluster]) if np.sum(same_cluster) > 1 else 0
        b_x = float('inf')
        for label in unique_labels:
            if label == labels[i] or label == -1: continue
            other_cluster = (labels == label)
            if np.sum(other_cluster) > 0:
                b_x = min(b_x, np.mean(dist_matrix[i, other_cluster]))
        if a_x == 0 and b_x == 0:
            s_x = 0
        else:
            s_x = (b_x - a_x) / max(a_x, b_x)
        silhouette_scores.append(s_x)
    silhouette_avg = np.mean(silhouette_scores) if silhouette_scores else -1
    print(f"Completed Silhouette Index: {silhouette_avg:.4f} (Time: {time.time() - start_time:.2f}s)")
    return silhouette_avg

# Mã hóa trực quan
def visual_encoding(dist_matrix, labels, traces, unique_activities):
    print("Running: Visual Encoding...")
    start_time = time.time()
    
    # 2.1 MDS
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42, normalized_stress='auto')
    coords = mds.fit_transform(dist_matrix)
    
    # 2.2 Giảm chồng lấp
    cluster_coords = {}
    for i, label in enumerate(labels):
        if label not in cluster_coords:
            cluster_coords[label] = []
        cluster_coords[label].append(coords[i])
    adjusted_coords = []
    for i, (x, y) in enumerate(coords):
        adjusted_coords.append([x + np.random.uniform(-0.1, 0.1), y + np.random.uniform(-0.1, 0.1)])
    adjusted_coords = np.array(adjusted_coords)
    
    # 2.3 Lấy mẫu dấu vết
    samples = {}
    for label in np.unique(labels):
        if label == -1: continue
        cluster_traces = [traces[i] for i in range(n_traces) if labels[i] == label]
        distances = [np.mean([dist_matrix[i, j] for j, l in enumerate(labels) if l == label]) for i in range(n_traces) if labels[i] == label]
        sorted_idx = np.argsort(distances)
        percentiles = [0, 25, 50, 75, 100]
        max_idx = len(sorted_idx) - 1
        samples[label] = [cluster_traces[sorted_idx[min(int(p * len(sorted_idx) / 100), max_idx)]] for p in percentiles]
    
    # 2.4 Sắp xếp dấu vết
    orders = {}
    for label in np.unique(labels):
        if label == -1: continue
        cluster_traces = [traces[i] for i in range(n_traces) if labels[i] == label]
        orders[label] = sorted(cluster_traces, key=len)
    
    # 2.5 Căn chỉnh dấu vết
    alignments = {}
    for label in np.unique(labels):
        if label == -1: continue
        cluster_traces = [traces[i] for i in range(n_traces) if labels[i] == label]
        medoid_idx = np.argmin([np.mean([dist_matrix[i, j] for j, l in enumerate(labels) if l == label]) for i in range(n_traces) if labels[i] == label])
        medoid = cluster_traces[medoid_idx]
        aligned = []
        for trace in cluster_traces:
            aligned_trace = list(trace)
            for i, act in enumerate(medoid):
                if i >= len(trace) or trace[i] != act:
                    aligned_trace.insert(i, "-")
            aligned.append(tuple(aligned_trace[:len(medoid)]))
        alignments[label] = aligned
    
    # 2.6 Process Graphs
    process_graphs = {}
    for label in np.unique(labels):
        if label == -1: continue
        cluster_log = pm4py.objects.log.obj.EventLog([trace for i, trace in enumerate(log) if labels[i] == label])
        net, im, fm = alpha_miner.apply(cluster_log)
        
        # Create list of nodes (only transitions)
        nodes = [t.label for t in net.transitions if t.label is not None]  # Lấy tất cả transitions có nhãn
        nodes = list(set(nodes))  # Loại bỏ trùng lặp
        nodes = [{"id": str(i), "label": node} for i, node in enumerate(nodes)]  # Tạo danh sách nodes với id và label
        
        # Create list of edges
        edges = []
        edge_counts = Counter()
        for trace in cluster_log:
            activities = [event["concept:name"] for event in trace]
            for i in range(len(activities) - 1):
                edge = (activities[i], activities[i + 1])
                edge_counts[edge] += 1
        
        # Map activity names to node IDs
        activity_to_id = {node["label"]: node["id"] for node in nodes}
        
        # Convert Petri Net to edges
        for place in net.places:
            in_transitions = [arc.source for arc in place.in_arcs if arc.source in net.transitions and arc.source.label is not None]
            out_transitions = [arc.target for arc in place.out_arcs if arc.target in net.transitions and arc.target.label is not None]
            for in_t in in_transitions:
                for out_t in out_transitions:
                    edge = (in_t.label, out_t.label)
                    weight = edge_counts.get(edge, 0)
                    if edge[0] in activity_to_id and edge[1] in activity_to_id:  # Chỉ thêm cạnh nếu cả hai hoạt động đều có trong nodes
                        edges.append({
                            "from": activity_to_id[edge[0]],
                            "to": activity_to_id[edge[1]],
                            "weight": weight,
                            "branch_label": None
                        })
        
        process_graphs[label] = {
            "nodes": nodes,
            "edges": edges
        }
    
    print(f"Completed Visual Encoding: {time.time() - start_time:.2f}s")
    return adjusted_coords, samples, orders, alignments, process_graphs

# Tạo dữ liệu JSON
def create_json_data(labels, coords, samples, orders, alignments, process_graphs, unique_activities, silhouette_score):
    data = {
        "clusters": {},
        "legend": {act: idx for idx, act in enumerate(unique_activities)},
        "cluster_info": {
            "n_clusters": len(np.unique(labels)) - (1 if -1 in labels else 0),
            "rating": int(silhouette_score * 5)
        }
    }
    for label in np.unique(labels):
        if label == -1: continue
        cluster_traces = [traces[i] for i in range(n_traces) if labels[i] == label]
        activity_counts = Counter([act for trace in cluster_traces for act in trace])
        total_acts = sum(activity_counts.values())
        activity_dist = {act: count / total_acts for act, count in activity_counts.items()}
        performance = [len(trace) for trace in cluster_traces]
        performance_quartiles = np.percentile(performance, [25, 50, 75, 100])
        
        data["clusters"][str(label)] = {
            "coords": coords[labels == label].tolist(),
            "samples": [list(s) for s in samples[label]],
            "traces": [list(t) for t in orders[label]],
            "aligned_traces": [list(t) for t in alignments[label]],
            "size": len([l for l in labels if l == label]),
            "activity_distribution": activity_dist,
            "performance": performance_quartiles.tolist(),
            "process_graph": process_graphs[label]
        }
    with open("RequestForPayment.json", "w") as f:
        json.dump(data, f)
    return data


sub_scores = compute_substitution_scores_multi_gpu(traces, unique_activities)
ins_scores = compute_insertion_scores_multi_gpu(traces, unique_activities)
dist_matrix = generic_edit_distance_matrix(traces, sub_scores, ins_scores, unique_activities)
labels, n_clusters = cluster_traces(dist_matrix)
silhouette_score = compute_silhouette_index(dist_matrix, labels)
coords, samples, orders, alignments, process_graphs = visual_encoding(dist_matrix, labels, traces, unique_activities)
json_data = create_json_data(labels, coords, samples, orders, alignments, process_graphs, unique_activities, silhouette_score)
print("JSON data exported to visualization_data.json")

Running: Using 2 GPU(s)...
Running: Loading .xes file...


parsing log, completed traces ::   0%|          | 0/6886 [00:00<?, ?it/s]

Completed loading: 2.23s (Traces: 6886)
Running: Computing Substitution Scores (Algorithm 1)...
Completed Substitution Scores: 1.49s
sub_scores: min=-11.1204, max=25.2616, mean=0.4545
Running: Computing Insertion Scores (Algorithm 2)...
Completed Insertion Scores: 1.50s
ins_scores: min=-1.6096, max=19.5603, mean=0.9133
Running: Computing Generic Edit Distance matrix on multi-GPU (A4)...
A4 Distance Matrix: Processed 1/1 chunks (100.0%), ETA: 0.00s
Completed Distance Matrix: 1.08s
Running: DBSCAN clustering...
Completed DBSCAN: 0.86s (Clusters: 33)
Running: Computing Silhouette Index...
Completed Silhouette Index: 1.0000 (Time: 8.63s)
Running: Visual Encoding...
Completed Visual Encoding: 1062.81s
JSON data exported to visualization_data.json
